# Task 4.4.0
Implement the DSPy Module: Create a DSPy module that uses the strategy you devise to generate a cooperative answer.

# Design plan

1) Define DSPy Signatures
Step 1 (Interpretation): history, current Q, context → history, current Q, context, student_goal, pragmatic_need
Step 2 (Retrieval): expanded_query, conversation_summary → refined_context
Step 3 (Answering): history, current Q, summary, goal, need, refined_context → cooperative_answer

2) Implement Retriever
Index HTML sources.
Retrieval cascade: BM25 (topic folder) → ColBERTv2/FAISS (fine passages).

3) Build Pipeline Forward Pass
Step 1 → Step 2 → Step 3.
Add loop if retrieval weak (multi-hop).

4) Prepare Data
Load PragmatiCQA train/val/test JSONL.
Convert each round to DSPy example.
Truncate long histories.

5) Train & Run
Few-shot or supervised training on train set.
Run pipeline on val/test sets.


# We need to explain why we chose this strategy
# And reduce the slowness if possible!

In [1]:
# Configure the LLM Model

# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
import dspy
import os
from dotenv import load_dotenv
load_dotenv("../grok_key.ini",override=True)
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
dspy.configure(lm=lm)

In [2]:
class ToMQuery(dspy.Signature):
    """
    Signature that enriches a user query with Theory of Mind insights.
    """
    history = dspy.InputField(description="Previous conversation history of the user")
    current_question = dspy.InputField(description="The user's current question")
    context = dspy.InputField(description="Relevant context passages retrieved from the knowledge base")

    student_goal = dspy.OutputField(description="Summary of the user's learning goal or intent")
    pragmatic_need = dspy.OutputField(description="Underlying cooperative/pragmatic need inferred from the question")



class CooperativeAnswer(dspy.Signature):
    """
    Signature for generating a cooperative answer to the user's question.
    This module uses the enriched Theory of Mind information along with the
    retrieved context to produce a pragmatic and anticipatory answer.
    """
    history = dspy.InputField(
        description="Previous conversation history of the user, used for context and continuity"
    )
    current_question = dspy.InputField(
        description="The current question posed by the user"
    )
    context = dspy.InputField(
        description="Relevant context passages retrieved from the knowledge base or search module"
    )
    student_goal = dspy.InputField(
        description="Summary of the user's goal or interests, inferred from the conversation history"
    )
    pragmatic_need = dspy.InputField(
        description="Underlying cooperative or pragmatic need inferred from the user's question"
    )

    cooperative_answer = dspy.OutputField(
        description="Based on the input fields, generates a pragmatic and cooperative answer that anticipates potential follow-up questions"
    )


In [15]:
class CooperativeQAPipeline(dspy.Module):
    """
    Pipeline module that retrieves relevant context, enriches the query with
    Theory of Mind information, and generates a cooperative answer.
    """
    def __init__(self):
        super().__init__()
        # self.enrich_query = dspy.ChainOfThought(ToMQuery)
        # self.generate_answer = dspy.ChainOfThought(CooperativeAnswer)
        print("With Predict!")
        self.enrich_query = dspy.Predict(ToMQuery)
        self.generate_answer = dspy.Predict(CooperativeAnswer)

    def forward(self, current_question, history=""):
        # Step 1: Retrieve context (consider limiting passages for large datasets)
        retrieved_context = search(current_question).passages
        print("@@@@@ found the context. proceeding.. @@@@")

        # Step 2: Enrich the query with Theory of Mind information
        tom_enrichment = self.enrich_query(
            context=retrieved_context,
            current_question=current_question,
            history=history
        )
        print(" @@@@ completed the TOM enrichment. proceeding.. @@@")

        # Step 3: Generate cooperative answer using context + ToM enrichment
        print("@@@ generating the final answer.... @@@@")
        return self.generate_answer(
            history=history,
            current_question=current_question,
            context=retrieved_context,
            student_goal=tom_enrichment.student_goal,
            pragmatic_need=tom_enrichment.pragmatic_need
        )

In [4]:
# import bm25s
# import Stemmer
# import os

# # Original folder names
# folder_names = os.listdir("../PragmatiCQA-sources")

# stemmer = Stemmer.Stemmer("english")

# # Tokenize each folder name
# corpus_tokens = bm25s.tokenize(folder_names, stopwords="en", stemmer=stemmer)

# # Keep track of positions
# retriever = bm25s.BM25(k1=0.9, b=0.4)
# retriever.index(corpus_tokens)

# def search(query: str, k: int):
#     tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
#     results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    
#     # 'results' is a list of token lists, so we can match them to the original folder names
#     for i, token_list in enumerate(results):
#         for j, ct in enumerate(corpus_tokens):
#             if ct == token_list:
#                 folder_name = folder_names[j]
#                 break
#         else:
#             folder_name = "UNKNOWN"
#         print(f"result: {folder_name} , score: {scores[i]}")

# search("Harry", 3)


In [5]:
# utility function

# Traverse a directory and read html files - extract text from the html files
import os
from bs4 import BeautifulSoup
import re

def read_html_files_clean(directory):
    
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                text = soup.get_text()
                # remove empty lines
                text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
                # optional: collapse multiple spaces
                text = re.sub(r'\s+', ' ', text)
                texts.append(text)
    return texts


In [6]:
# %pip install -U faiss-cpu  # or faiss-gpu if you have a GPU

In [7]:
import os
folder_names = os.listdir("../PragmatiCQA-sources")
corpus = []
# counter = 0 
for folder in folder_names:
    # files = os.listdir("../PragmatiCQA-sources/" + folder)
    texts = read_html_files_clean("../PragmatiCQA-sources/" + folder)
    corpus.extend(texts)
    # counter += len(texts)
    print(f"added {len(texts)} files from {folder}")
    print("the total number of loaded files is: ", len(corpus))
    # if counter > 3000:
    #     break
    
#     folder_texts = read_html_files(folder)
#     corpus.extend(folder_texts)





added 449 files from 'Cats' Musical
the total number of loaded files is:  449
added 250 files from A Nightmare on Elm Street
the total number of loaded files is:  699
added 499 files from Arrowverse
the total number of loaded files is:  1198
added 492 files from Barney
the total number of loaded files is:  1690
added 498 files from Baseball
the total number of loaded files is:  2188
added 496 files from Batman
the total number of loaded files is:  2684
added 265 files from Big Nate
the total number of loaded files is:  2949
added 470 files from Bleach
the total number of loaded files is:  3419
added 310 files from Britney Spears
the total number of loaded files is:  3729
added 282 files from Detective Conan
the total number of loaded files is:  4011
added 499 files from Dinosaur
the total number of loaded files is:  4510
added 36 files from Doctor Who
the total number of loaded files is:  4546
added 322 files from Doom Patrol
the total number of loaded files is:  4868
added 61 files fr

In [8]:
max_characters = 10000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 3  # number of documents to retrieve per search query

from sentence_transformers import SentenceTransformer

# Load an extremely efficient local model for retrieval
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")

# Create an embedder using the model's encode method
embedder = dspy.Embedder(model.encode)

search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Training a 32-byte FAISS index with 338 partitions, based on 28573 x 1024-dim embeddings


In [9]:
test = search("What year is it today?").passages
print(test[0])
import pprint
pprint.pprint(test[0])

Decades and years 20th century 1900s 1910s 1920s 1930s 1940s 1950s 1960s 1970s 1980s 1990s 20th century : 1900s 1900 - 1901 - 1902 - 1903 - 1904 - 1905 - 1906 - 1907 - 1908 - 1909 20th century : 1910s 1910 - 1911 - 1912 - 1913 - 1914 - 1915 - 1916 - 1917 - 1918 - 1919 20th century : 1920s 1920 - 1921 - 1922 - 1923 - 1924 - 1925 - 1926 - 1927 - 1928 - 1929 20th century : 1930s 1930 - 1931 - 1932 - 1933 - 1934 - 1935 - 1936 - 1937 - 1938 - 1939 20th century : 1940s 1940 - 1941 - 1942 - 1943 - 1944 - 1945 - 1946 - 1947 - 1948 - 1949 20th century : 1950s 1950 - 1951 - 1952 - 1953 - 1954 - 1955 - 1956 - 1957 - 1958 - 1959 20th century : 1960s 1960 - 1961 - 1962 - 1963 - 1964 - 1965 - 1966 - 1967 - 1968 - 1969 20th century : 1970s 1970 - 1971 - 1972 - 1973 - 1974 - 1975 - 1976 - 1977 - 1978 - 1979 20th century : 1980s 1980 - 1981 - 1982 - 1983 - 1984 - 1985 - 1986 - 1987 - 1988 - 1989 20th century : 1990s 1990 - 1991 - 1992 - 1993 - 1994 - 1995 - 1996 - 1997 - 1998 - 1999
('Decades and years

In [19]:
## "FOR THE FIRST QUESTION OF EACH CONVERSATION ONLY"

# load the dataset

# Load jsonl from dataset directory
import json
import os  

def read_data(filename, dataset_dir="../PragmatiCQA/data"):
    corpus = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            corpus.append(json.loads(line))
    return corpus

pcqa_test = read_data("val.jsonl")
# print()
# print(f"Loaded {len(pcqa_test)} documents from PragmatiCQA val set.")

all_questions = [pcqa_test[i]['qas'][j]['q'] for i in range(len(pcqa_test)) for j in range(len(pcqa_test[i]['qas']))]
print(len(all_questions))

first_questions_only = [pcqa_test[i]['qas'][0]['q'] for i in range(len(pcqa_test))]

# Pretty print the first document to check the structure
# import pprint
# print("the total number of first questions is: ", len(first_questions_only)) 
# for q in first_questions_only:
#     pprint.pprint(q)


1526


In [16]:
rag = CooperativeQAPipeline()  # create once

# now just call it multiple times
answer1 = rag("What is the main plot of The Legend of Zelda?")
# answer2 = rag("Who is the main villain in Zelda?")
# answer3 = rag("What year was Zelda first released?")

print(answer1)
# print(answer2.response)
# print(answer3.response)

With Predict!
@@@@@ found the context. proceeding.. @@@@


2025/08/21 20:09:19 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/21 20:09:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


 @@@@ completed the TOM enrichment. proceeding.. @@@
@@@ generating the final answer.... @@@@
Prediction(
    cooperative_answer='The main plot of The Legend of Zelda series centers on Link, a courageous hero, who embarks on epic quests to save Princess Zelda and the kingdom of Hyrule from various threats, most notably the evil sorcerer Ganon (or Ganondorf). The story typically revolves around the Triforce—a powerful artifact split into three pieces representing Courage (held by Link), Wisdom (held by Zelda), and Power (often sought by Ganon). In the original game, The Legend of Zelda, Link must collect fragments of the Triforce of Wisdom, defeat Ganon, and rescue Zelda from his lair. Across the series, this core narrative expands into different timelines, with Link facing new challenges, exploring vast worlds, and using magical items like the Master Sword to restore peace.\n\nTo make this more engaging for your interest in gaming or building foundational knowledge, I recommend startin

In [12]:
dspy.inspect_history()





[2025-08-21T20:07:01.705773]

System message:

Your input fields are:
1. `history` (str): Previous conversation history of the user, used for context and continuity
2. `current_question` (str): The current question posed by the user
3. `context` (str): Relevant context passages retrieved from the knowledge base or search module
4. `student_goal` (str): Summary of the user's goal or interests, inferred from the conversation history
5. `pragmatic_need` (str): Underlying cooperative or pragmatic need inferred from the user's question
Your output fields are:
1. `reasoning` (str): 
2. `cooperative_answer` (str): Based on the input fields, generates a pragmatic and cooperative answer that anticipates potential follow-up questions
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## history ## ]]
{history}

[[ ## current_question ## ]]
{current_question}

[[ ## context ## ]]
{context}

[[ ## stud

In [13]:
last_call = dspy.inspect_history(1)
print(last_call)





[2025-08-21T20:07:01.705773]

System message:

Your input fields are:
1. `history` (str): Previous conversation history of the user, used for context and continuity
2. `current_question` (str): The current question posed by the user
3. `context` (str): Relevant context passages retrieved from the knowledge base or search module
4. `student_goal` (str): Summary of the user's goal or interests, inferred from the conversation history
5. `pragmatic_need` (str): Underlying cooperative or pragmatic need inferred from the user's question
Your output fields are:
1. `reasoning` (str): 
2. `cooperative_answer` (str): Based on the input fields, generates a pragmatic and cooperative answer that anticipates potential follow-up questions
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## history ## ]]
{history}

[[ ## current_question ## ]]
{current_question}

[[ ## context ## ]]
{context}

[[ ## stud